In [4]:
import os
import sys

import pandas as pd

sys.path.append("/home/sergey/drclinics/common")
sys.path.append("/home/sergey/drclinics/reports")

import datetime
from utils import get_path, DATETIME_FORMAT

from universal_connection import UniversalConnection, DBType
import numpy as np
import hashlib

from report_utils import query_by_name

In [5]:
conn = UniversalConnection('../../../../.credentials/telemed/prom.cfg', DBType.Postgres)
df_web: pd.DataFrame = query_by_name(conn, os.getcwd(), 'web')
conn.close()

2020-07-14 14:25:58 connect to postgres database using config file "../../../../.credentials/telemed/prom.cfg"
2020-07-14 14:25:58 creating ssh tunnel to 172.16.100.19 as root...
2020-07-14 14:25:58 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 34305
2020-07-14 14:25:58 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-07-14 14:25:58 @telemed query:
                    with temp as (
				    select 	a.id,
				    		a.patient_id,
				    		pe.phone,
				    		product.full_name 
				    --
				    from appointment a 
				    --
				    	inner join patient pa
				    	   on pa.id = a.patient_id
				    	inner join person pe
				    	   on pa.person_id = pe.id
				    --
				    	inner join promotion p
				    	  on a.promotion_id = p.id
				    	inner join product_condition pc
				    	  on pc.id = p.product_condition_id
				    --
				    	in

In [13]:
conn = UniversalConnection('../../../../.credentials/telemed/prom.cfg', DBType.Postgres)
df_partners: pd.DataFrame = query_by_name(conn, os.getcwd(), 'partners-no-cons')
conn.close()

2020-07-14 15:35:14 connect to postgres database using config file "../../../../.credentials/telemed/prom.cfg"
2020-07-14 15:35:14 creating ssh tunnel to 172.16.100.19 as root...
2020-07-14 15:35:15 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 41703
2020-07-14 15:35:15 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-07-14 15:35:15 @telemed query:
                    with temp as (
				    select 	partner."name",
				    		promotion.id,
				    		pp.patient_id 
				    --
				    from partner
				    --
				    inner join promotion on promotion.partner_id = partner.id
				    inner join patient_promotion pp on pp.promotion_id  = promotion.id 
				    --
				    where partner.id in (6, 8, 1061)
				    )
				    --------------------------------------------------
				    select 	patient.id patient_id,
				    		person.phone,
				    		promo

In [14]:
df_web['partner'] = 'WEB'

In [15]:
df_web

,patient_id,phone,partner
0,3863288,79014274809,WEB
1,3749906,79169287282,WEB
2,3562456,79165149564,WEB
3,3588058,79535473467,WEB
4,3775187,79109216266,WEB
...,...,...,...
35938,2567727,79100999407,WEB
35939,3583640,79116055146,WEB
35940,4666417,79221949409,WEB
35941,3780654,79581445629,WEB


In [17]:
df_partners.drop(['start_', 'end_', 'partner_id'], axis=1)

,patient_id,phone,partner
0,29989,79112663622,Райффайзен Лайф
1,38035,79860000001,"ООО ""СК «Согласие-Вита»"
2,29989,79112663622,Райффайзен Лайф
3,38035,79860000001,"ООО ""СК «Согласие-Вита»"
4,64639,79137707747,"ООО ""СК «Согласие-Вита»"
...,...,...,...
1077169,4926989,79633660013,Русский Стандарт Страхование
1077170,4926989,79633660013,Русский Стандарт Страхование
1077171,4926989,79633660013,Русский Стандарт Страхование
1077172,4926989,79633660013,Русский Стандарт Страхование


In [19]:
pd.concat([df_web, df_partners.drop(['start_', 'end_', 'partner_id'], axis=1)]).reset_index(drop=True)

,patient_id,phone,partner
0,3863288,79014274809,WEB
1,3749906,79169287282,WEB
2,3562456,79165149564,WEB
3,3588058,79535473467,WEB
4,3775187,79109216266,WEB
...,...,...,...
1113112,4926989,79633660013,Русский Стандарт Страхование
1113113,4926989,79633660013,Русский Стандарт Страхование
1113114,4926989,79633660013,Русский Стандарт Страхование
1113115,4926989,79633660013,Русский Стандарт Страхование


In [20]:
total_patients = pd.concat([df_web, df_partners.drop(['start_', 'end_', 'partner_id'], axis=1)]).reset_index(drop=True)

In [25]:
total_patients.groupby(['patient_id', 'phone', 'partner'])[['patient_id']].count().rename(columns={'patient_id':'subs'}).reset_index()

,patient_id,phone,partner,subs
0,611,79067166882,WEB,1
1,1038,79614598975,WEB,1
2,1093,79831649942,WEB,1
3,1781,79035092518,Альфа Страхование,2
4,1816,79319667076,СК Согласие,4
...,...,...,...,...
329148,4927003,79117860032,Русский Стандарт Страхование,1
329149,4927004,79037181891,Русский Стандарт Страхование,1
329150,4927005,79600448546,Русский Стандарт Страхование,1
329151,4927006,79139859198,Русский Стандарт Страхование,1


In [22]:
len(total_patients.phone.unique())

303571

In [26]:
total_patients_grouped = total_patients.groupby(['patient_id', 'phone', 'partner'])[['patient_id']].count().rename(columns={'patient_id':'subs'}).reset_index()

In [27]:
total_patients_grouped

,patient_id,phone,partner,subs
0,611,79067166882,WEB,1
1,1038,79614598975,WEB,1
2,1093,79831649942,WEB,1
3,1781,79035092518,Альфа Страхование,2
4,1816,79319667076,СК Согласие,4
...,...,...,...,...
329148,4927003,79117860032,Русский Стандарт Страхование,1
329149,4927004,79037181891,Русский Стандарт Страхование,1
329150,4927005,79600448546,Русский Стандарт Страхование,1
329151,4927006,79139859198,Русский Стандарт Страхование,1


In [28]:
def correct_phone(phone):
    try:
        if len(phone) == 11 and phone[0] == '7':
            return phone
        elif len(phone) == 11 and phone[0] == '8':
            return '7' + phone[1:]
        elif len(phone) == 10 and phone[0] == '9':
            return '7' + phone
        elif len(phone) == 10 and phone[0] == '4':
            return '7' + phone
        return np.nan
    except:
        return np.nan


In [43]:
df_hashed = pd.DataFrame(total_patients_grouped.phone.unique()).rename(columns={0:'phone'})

In [44]:
df_hashed = df_hashed.phone.apply(correct_phone)

In [45]:
df_hashed = pd.DataFrame(df_hashed)

In [46]:
df_hashed = df_hashed[~df_hashed.phone.isna()].reset_index(drop=True)

In [47]:
df_hashed

,phone
0,79067166882
1,79614598975
2,79831649942
3,79035092518
4,79319667076
...,...
303125,79117860032
303126,79037181891
303127,79600448546
303128,79139859198


In [48]:
def make_hash_MD5(s):
    return hashlib.md5(s.encode('utf-8')).hexdigest()

In [49]:
df_hashed['hashed_phone'] = df_hashed['phone'].apply(make_hash_MD5)

In [50]:
df_hashed

,phone,hashed_phone
0,79067166882,33f4fa64a760d0ea37a6ae09c476efeb
1,79614598975,47780488302c0ffd693046dcc9f435db
2,79831649942,03a14e4c26570232ad002b2365bf05f1
3,79035092518,faae64e7afb1bda717706891a9f725c0
4,79319667076,3e2bcc3bfff71f8e4dd12dfe421f25ef
...,...,...
303125,79117860032,ff3377092246444603fe8bdf5f8301bd
303126,79037181891,b921c52ec6dbee90e35abc2e1ea5fe3b
303127,79600448546,d799f05c8824f54dbb7d3d1384c23c47
303128,79139859198,1b5c469f27a73e0d62db6b4bd4cba57d


In [52]:
len(df_hashed.hashed_phone.unique())

302843

In [53]:
len(df_hashed.phone.unique())

302843

In [2]:
adhoc = pd.read_csv('adhoc453_res.csv')

In [55]:
adhoc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5055473 entries, 0 to 5055472
Data columns (total 2 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   msisdn_hash  object
 1   ident_check  int64 
dtypes: int64(1), object(1)
memory usage: 77.1+ MB


In [58]:
len(adhoc.msisdn_hash.unique())

4583378

In [51]:
adhoc.head()

,msisdn_hash,ident_check
0,00002f61304269d06fa27d995367c1cf,0
1,00004bf28b7cf3786b32d6e2a3b5b037,0
2,00006d174ed35baaa2ecf57ee89ee459,0
3,0000b9043f224011cc5f5813912c193d,0
4,00010efc90a2644131ae348f930d1896,1


In [54]:
adhoc[adhoc['msisdn_hash'].isin(list(df_hashed.hashed_phone.unique()))]

,msisdn_hash,ident_check
11,0002a363f579751718c1253f2cac17c7,0
51,00094a50d6e2f6c033c708b83acdd769,0
55,000b4f345bef64d0e0ee62a2766d9e16,0
66,000dfb9decce7665686d370bc741c9f5,0
67,000dfb9decce7665686d370bc741c9f5,0
...,...,...
5055329,ffd72346c0a526396043a00f6f02ca35,0
5055345,ffdc3dea7f8807e9c449a44e8b1812ef,1
5055414,fff1304390805e5a5a663a324d3f6119,1
5055431,fff5c634cb662daf85d8f0fd1850f81d,1


In [59]:
same_hash = adhoc[adhoc['msisdn_hash'].isin(list(df_hashed.hashed_phone.unique()))]

In [60]:
same_hash['ident_check_2'] = 1

In [61]:
same_hash

,msisdn_hash,ident_check,ident_check_2
11,0002a363f579751718c1253f2cac17c7,0,1
51,00094a50d6e2f6c033c708b83acdd769,0,1
55,000b4f345bef64d0e0ee62a2766d9e16,0,1
66,000dfb9decce7665686d370bc741c9f5,0,1
67,000dfb9decce7665686d370bc741c9f5,0,1
...,...,...,...
5055329,ffd72346c0a526396043a00f6f02ca35,0,1
5055345,ffdc3dea7f8807e9c449a44e8b1812ef,1,1
5055414,fff1304390805e5a5a663a324d3f6119,1,1
5055431,fff5c634cb662daf85d8f0fd1850f81d,1,1


In [62]:
same_hash.to_csv('maxima-telecom-MD5.csv', index=False)